In [36]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [37]:
from pathlib import Path
from src.utils.config_loader import load_config

base_dir = Path(os.getcwd()).parent

config = load_config(base_dir / 'secrets.yaml')

In [38]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import (SystemMessage, HumanMessage)

os.environ['OPENAI_API_VERSION'] = config['OPENAI_API_VERSION']
os.environ['AZURE_OPENAI_ENDPOINT'] = config['OPENAI_API_BASE']
os.environ['AZURE_OPENAI_API_KEY'] = config['OPENAI_API_KEY']

llm = AzureChatOpenAI(
    deployment_name=config['OPENAI_DEPLOYMENT_NAME'],
    temperature=0.0
)

# TODO: Maybe for this task i should experiment with temperature

In [39]:
import json

with open(base_dir / 'data/val_set.json', 'r') as f:
    data = json.load(f)

In [ ]:
from tqdm import tqdm

results = []

data_sample = data #data[:1]

for item in tqdm(data_sample):

    version_1 = item['version_1']
    version_2 = item['version_2']
    ref_summary = item['ref_summary']

    msg = [
        SystemMessage(
            content=(
                "You are an urban planning and regulatory documentation expert, "
                "specializing in Norwegian zoning plans. Your role is to assist "
                "case workers by identifying and clearly summarizing the differences "
                "between two versions of a zoning plan text. The summary should be as brief "
                "as possible, yet clear and informative. Avoid unnecessary details and write "
                "the summary in Norwegian. If version 1 contains text, but not version 2, "
                "it means the text has been removed. If it's the other way around, the "
                "text has been added. Do not refer to version 1 or version 2 in the "
                "summary. Only describe what has been removed, added, or changed."
            )
        ),
        HumanMessage(
            content=f"""\
Summarize the differences between the following two versions:

Version 1:
{version_1}

Version 2:
{version_2}

Summary: 
"""
    )
]

    try: 
        response = llm.invoke(msg)

        results.append({
            'id': item['id'],
            'ref_summary': ref_summary,
            'model_summary': response.content
        })

    except Exception as e:
        print(f"Error: {e}")
        continue

100%|██████████| 35/35 [03:02<00:00,  5.23s/it]


In [41]:
with open(base_dir / f"results/{config['OPENAI_DEPLOYMENT_NAME']}_ZEROSHOT_test_4.jsonl", "w") as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')